In [ ]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
from tqdm.notebook import tqdm
from torchvision.utils import save_image
from torchvision.utils import make_grid
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
DEVICE = torch.device('cuda')

In [ ]:
os.chdir('/home/kamenev_v/Deepfake_proj/DeepFakeDetection/')

In [ ]:
from lib.data.dataset import *
from lib.models.PFD import *
from lib.utils import *
from lib.training.train_model import *
from lib.models.FTFD import *
from lib.models.BaseDetector import *
from lib.training.trainerold import train_old

In [ ]:
batch_size = 20
image_size = 224

In [ ]:
TRAIN = pd.read_csv('lib/data/FINAL_TRAIN_P2.csv')
TEST = pd.read_csv('lib/data/FINAL_TEST_P2.csv')
# TRAIN['path']=TRAIN['path'].apply(lambda x: 'lib/data/MixedData/'+x)
# TEST['path']=TEST['path'].apply(lambda x: 'lib/data/MixedData/'+x)
TRAIN['class'] = TRAIN['class'].apply(lambda x: torch.tensor(x,dtype=torch.float32))
TEST['class'] = TEST['class'].apply(lambda x: torch.tensor(x,dtype=torch.float32))

## Data Loading&Preparation

In [ ]:
def patch_apply(x):
    return torch.tensor([[x,x],[x,x]],dtype=torch.float32)

In [ ]:
TRAIN

,path,class
0,lib/data/MixedData/True/simswappic_107223.jpg,tensor(0.)
1,lib/data/MixedData/True/01687.png,tensor(0.)
2,lib/data/MixedData/Fake/newpic_977.jpgresult.jpg,tensor(1.)
3,lib/data/MixedData/Fake/3981.jpg,tensor(1.)
4,lib/data/MixedData/Fake/266.jpg,tensor(1.)
...,...,...
12271,lib/data/CUSTOMV2/Fake/id01637_1935__video1_au...,tensor(1.)
12272,lib/data/MixedData/Fake/swappicm_558.jpgresult...,tensor(1.)
12273,lib/data/MixedData/Fake/newpicm_735.jpgresult.jpg,tensor(1.)
12274,lib/data/MixedData/Fake/newpicm_598.jpgresult.jpg,tensor(1.)


In [ ]:
TRAIN['class'] = TRAIN['class'].apply(lambda x: patch_apply(x))
TEST['class'] = TEST['class'].apply(lambda x: patch_apply(x))
train_dataset = DeepFakeDataset(TRAIN)
test_dataset = DeepFakeDataset(TEST)

In [ ]:
TRAIN

,path,class
0,lib/data/MixedData/True/simswappic_107223.jpg,tensor(0.)
1,lib/data/MixedData/True/01687.png,tensor(0.)
2,lib/data/MixedData/Fake/newpic_977.jpgresult.jpg,tensor(1.)
3,lib/data/MixedData/Fake/3981.jpg,tensor(1.)
4,lib/data/MixedData/Fake/266.jpg,tensor(1.)
...,...,...
12271,lib/data/CUSTOMV2/Fake/id01637_1935__video1_au...,tensor(1.)
12272,lib/data/MixedData/Fake/swappicm_558.jpgresult...,tensor(1.)
12273,lib/data/MixedData/Fake/newpicm_735.jpgresult.jpg,tensor(1.)
12274,lib/data/MixedData/Fake/newpicm_598.jpgresult.jpg,tensor(1.)


In [ ]:
model = EffNetb1(patch=False,pretrained_folder = 'lib/models/b1_patch.pth')
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())
loss = torch.nn.BCELoss()
res = train_old(model,optimizer,loss,50,train_dataset,test_dataset,False,'lib/models/b1_default.pth')

In [ ]:
from lib.metrics.evaluate import Evaluator
checker = Evaluator(type_of_model='av',loss_fn=nn.BCELoss(),patch_v=True)
checker.estimate_dataset(test_dataset,model)

ValueError: too many values to unpack (expected 3)